<img src="img/hector_scene.png" width="700" />

<img src="img/robotignite_logo_text.png" width="700" />

# Programming Drones

## Unit 3: Adapting the PTAM framework for another drone

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>2 hours</b><br><br>
In this unit, you are going to see how you can adapt the camera-based SLAM system you saw in the previous unit, which was created for the Parrot AR Drone, to another drone.

<p style="background:green;color:white;">END OF SUMMARY</p>

In the previous unit, you learned how to work with the **tum_ardrone** package in order to perform camera-based Navigation. But that package is meant to only work with the Parrot AR Drone, which is kind of limited. So, in this unit, what I want to explain to you is how you can adapt this package to work with other nodes. You are not going to do a 100% adaptation, but you are going to learn the basic stuff you need to know in order to do it.

So, as may have already noticed, the simulation and the drone you are going to use in this unit are different. For this unit, you are going to use the **hector_quadrotor** drone. It is a very basic drone model simulation, with all the basic sensors we need, like a camera, an imu, etc. You can check the simulation repository in the following link: https://github.com/tu-darmstadt-ros-pkg/hector_quadrotor

<img src="img/hector_quadrotor.png" width="500" />

Now that I have introduced you to the new drone, let's begin with the unit! The adaptation process will be divided in four main parts, which are the following:

* Download the code

* Modify the tum_ardrone package

* Publish Navdata

* Additional Steps

So... let's stop the talking and begin with the real work!

### Downloading the code

In the previous unit, you were working with a preinstalled and preconfigured version of the **tum_ardrone** package. But this version is not accessible to students. So, if we want to modify this package in order to adapt it to other drones, the first thing we'll need to do is download it.

First, go to the **src** folder in your workspace.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roscd;
cd ..;
cd src;

You can download the package using the followng command:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
git clone https://github.com/tum-vision/tum_ardrone.git

Also, after downloading the package, make sure that the branch you downloaded is the **indigo-devel**. You can check that by going to the downloaded package and executing the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
git checkout indigo-devel

<img src="img/tum_indigo_devel.png"width="800"/>

You will also need to download the **ardrone_autonomy** package. You will need this package in order to be able to publish the rewired navdata used by the **tum_ardrone** package. You will get deeper into this topic later. To download this package, you can execute the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roscd;
cd ..;
cd src;

In [ ]:
git clone https://github.com/AutonomyLab/ardrone_autonomy.git

In the end, you should have both packages in your workspace, like this:

<img src="img/tum_download_pkgs.png" width="400" />

### Modifying the tum_ardrone package

So, now it's time to start the modifications. There are many topics that can be used by this package, but for this adaptation, we are going to focus only on the main ones. These topics are the following:

* A topic for where to send velocity commands to the drone. In this simulation (and usually always), this topic is named **cmd_vel**.


* A topic for where to read data from the camera. In this simulation, this topic is named **/front_cam/camera/image**.


* Two topics for taking off and landing the drone. In this simulation, these topics are named **/takeoff** and **/land**.


* A topic for where all the navigation data will be published. This topic is known as the **navdata** topic, and unfortunately, this is something that is only provided in the Parrot AR Drone simulation. So, for this simulation, we'll need to make a workaround in order to have this topic. But don't worry about this now, we'll see it later.

So, basically, the work we are going to do with the **tum_ardrone** package will be to remap all the default topics for the AR Drone simulation to the equivalent topics in this simulation. Easy, right?

For this task, you'll need to focus on three files:

* **RosThread.cpp** (which is located in the UINode folder)


* **ControlNode.cpp** (which is located in the autopilot folder )


* **EstimationNode.cpp** (which is located in the stateestimation folder)

#### RosThread.cpp

Within the **RosThread::run()** function of this file, you'll need to find the following lines and modify them as shown below:

vel_pub	   = nh_.advertise<geometry_msgs::Twist>(nh_.resolveName("cmd_vel"),1);
vel_sub	   = nh_.subscribe(nh_.resolveName("cmd_vel"),50, &RosThread::velCb, this);

navdata_sub	   = nh_.subscribe(nh_.resolveName("hector/navdata"),50, &RosThread::navdataCb, this);

takeoff_pub	   = nh_.advertise<std_msgs::Empty>(nh_.resolveName("/takeoff"),1);
land_pub	   = nh_.advertise<std_msgs::Empty>(nh_.resolveName("/land"),1);

takeoff_sub	   = nh_.subscribe(nh_.resolveName("/takeoff"),1, &RosThread::takeoffCb, this);
land_sub	   = nh_.subscribe(nh_.resolveName("/land"),1, &RosThread::landCb, this);

#### ControlNode.cpp

Within the **ControlNode::ControlNode()** function of this file, you'll need to find the following lines and modify them as shown below:

In [ ]:
control_channel = nh_.resolveName("cmd_vel");
takeoff_channel = nh_.resolveName("/takeoff");
land_channel = nh_.resolveName("/land");

#### EstimationNode.cpp

Within the **EstimationNode::EstimationNode()** function of this file, you'll need to find the following lines and modify them as shown below:

In [ ]:
navdata_channel = nh_.resolveName("hector/navdata");
control_channel = nh_.resolveName("cmd_vel");
video_channel = nh_.resolveName("/front_cam/camera/image");

And that's it! You have already done the necessary work in order to adapt the **tum_ardrone** package for this simulation. Of course, there are more topics that you could remap. But, depending on the simulation you use, they will probably not be available, so you would have to publish those topics yourself (like we will do with the **navdata** topic). 

So, at this point, you are ready to compile the package in order to create all the libraries and executables, and to check that you haven't broken anything in the process. Also, you will compile the **ardrone_autnomy** package, which will be required for the next section. In order to compile your workspace, you can execute the following commands:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roscd;
cd ..;
catkin_make;

**NOTE**: Don't worry if some warnings appear during the compilation. This is normal. Just make sure that the compilation finishes and it is 100% completed.

### Publish Navdata

Great job! You are almost done! This one is the last step to finishing this adaptation, although it is also the most difficult. In this section, you are going to see how to create a node that publishes all the required **navdata** into a topic, which will be used by the **tum_ardrone** package.

But, first of all, let's have a look at what we need to publish into this topic. The type of message that we are going to publish into this topic is the following: **ardrone_autonomy/Navdata**. You can have a look at the structure of the message by executing the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
rosmsg show ardrone_autonomy/Navdata

You will get a structure like this one:

In [ ]:
std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
float32 batteryPercent
uint32 state
int32 magX
int32 magY
int32 magZ
int32 pressure
int32 temp
float32 wind_speed
float32 wind_angle
float32 wind_comp_angle
float32 rotX
float32 rotY
float32 rotZ
int32 altd
float32 vx
float32 vy
float32 vz
float32 ax
float32 ay
float32 az
uint8 motor1
uint8 motor2
uint8 motor3
uint8 motor4
uint32 tags_count
uint32[] tags_type
uint32[] tags_xc
uint32[] tags_yc
uint32[] tags_width
uint32[] tags_height
float32[] tags_orientation
float32[] tags_distance
float32 tm

Wait! Don't panic! It's not as huge as it seems, trust me. Most of the variables that are contained in this message can be overlooked, at least when working with a simulation. We are just going to focus on providing some of them.

So, let's go! The first thing you are going to do is create a new package.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roscd;
cd ..;
cd src;

In [ ]:
catkin_create_pkg publish_navdata

Within this package, create two new folders, named **src** and **launch**. Inside the src folder, create a file named **publish_navdata_code.py**; and inside the launch file, create a file named **publish_navdata.launch**. In the end, you should have a structure like this one.

<img src="img/navdata_pkg.png" width="400" />

Great! Now, let's copy the following code to the **publish_navdata_code.py** file. Later, I'll analyze the code and what each part of it does.

**publish_navdata_code.py**

In [ ]:
#! /usr/bin/env python

import rospy
from ardrone_autonomy.msg import Navdata
from sensor_msgs.msg import Imu, Range
from geometry_msgs.msg import PoseStamped, Twist

rotX = 0
rotY = 0
rotZ = 0
height = 0
vx = 0
vy = 0
vz = 0
ax = 0
ay = 0
az = 1
M_PI = 3.14159265358979323846264338327950288

def imu_callback(msg):
    global rotX
    global rotY
    global rotZ
    rotX = msg.orientation.x
    rotY = msg.orientation.y
    rotZ = msg.orientation.z
    
def pose_callback(msg):
    global height
    height = msg.pose.position.z
    
def vel_callback(msg):
    global vx
    global vy
    global vz
    vx = msg.linear.x
    vy = msg.linear.y
    vz = msg.linear.z


rospy.init_node('publish_navdata')

rate = rospy.Rate(2)

imu_sub = rospy.Subscriber('/raw_imu', Imu, imu_callback)
pose_sub = rospy.Subscriber('/ground_truth_to_tf/pose', PoseStamped, pose_callback)
vel_sub = rospy.Subscriber('/cmd_vel', Twist, vel_callback)

navdata_pub = rospy.Publisher('/hector/navdata', Navdata, queue_size=1)
navdata = Navdata()

while not rospy.is_shutdown():

    navdata.batteryPercent = 100
    navdata.rotX = rotX / M_PI * 180
    navdata.rotY = rotY / M_PI * 180
    navdata.rotZ = rotZ / M_PI * 180
    navdata.altd = height * 1000
    navdata.vx = vx
    navdata.vy = vy
    navdata.vz = vz
    navdata.ax = ax
    navdata.ay = ay
    navdata.az = az
    navdata.tm = rospy.get_time() * 1000000
    
    navdata.header.stamp = rospy.Time.now()
    navdata.header.frame_id = "base_link"
    navdata.state = 2
    navdata.magX = 0
    navdata.magY = 0
    navdata.magZ = 0
    navdata.pressure = 0
    navdata.temp = 0
    navdata.wind_speed = 0.0
    navdata.wind_angle = 0.0
    navdata.wind_comp_angle = 0.0
    navdata.tags_count = 0

    navdata_pub.publish(navdata)
    rate.sleep()

Let's analyze this code more deeply now.

In [ ]:
import rospy
from ardrone_autonomy.msg import Navdata
from sensor_msgs.msg import Imu, Range
from geometry_msgs.msg import PoseStamped, Twist

Here, you are just importing some necessary modules and messages that you will later use for the Publishers and Subscribers of the code.

In [ ]:
rotX = 0
rotY = 0
rotZ = 0
height = 0
vx = 0
vy = 0
vz = 0
ax = 0
ay = 0
az = 1
M_PI = 3.14159265358979323846264338327950288

Here, you are defining some global variables to make them easier to use through the code.

In [ ]:
def imu_callback(msg):
    global rotX
    global rotY
    global rotZ
    rotX = msg.orientation.x
    rotY = msg.orientation.y
    rotZ = msg.orientation.z
    
def pose_callback(msg):
    global height
    height = msg.pose.position.z
    
def vel_callback(msg):
    global vx
    global vy
    global vz
    vx = msg.linear.x
    vy = msg.linear.y
    vz = msg.linear.z
    
imu_sub = rospy.Subscriber('/raw_imu', Imu, imu_callback)
pose_sub = rospy.Subscriber('/ground_truth_to_tf/pose', PoseStamped, pose_callback)
vel_sub = rospy.Subscriber('/cmd_vel', Twist, vel_callback)

These are the Subscriber definitions and callbacks. Each time a message is published in one of these topics (**/raw_imu**, **/ground_truth_to_tf/pose**, and **/cmd_vel**), the related callbacks are called. Inside the callbacks, all you are doing is putting the values we are interested in into global variables. 

* From the **/imu_raw** topic, which contains the data from the Imu sensor, we get the values of the orientation of the drone.


* From the **/ground_truth_to_tf/pose** topic, which contains the data from the pose of the drone, we get the values of the z-axis, which is the altitude of the drone.


* From the **/cmd_vel** topic, which contains the data from the velocities sent to the drone, we get the values of all the linear components of these velocities.

In [ ]:
navdata_pub = rospy.Publisher('/hector/navdata', Navdata, queue_size=1)
navdata = Navdata()

Here, we are just defining a Publisher, indicating in what topic we are going to publish all this navdat data. In this case, the topic where we will publish this data will be named **/hector_navdata**. Also, we created an instance of the **ardrone_autonomy/Navdata** message.

In [ ]:
while not rospy.is_shutdown():

    navdata.batteryPercent = 100
    navdata.rotX = rotX / M_PI * 180
    navdata.rotY = rotY / M_PI * 180
    navdata.rotZ = rotZ / M_PI * 180
    navdata.altd = height * 1000
    navdata.vx = vx
    navdata.vy = vy
    navdata.vz = vz
    navdata.ax = ax
    navdata.ay = ay
    navdata.az = az
    navdata.tm = rospy.get_time() * 1000000
    
    navdata.header.stamp = rospy.Time.now()
    navdata.header.frame_id = "base_link"
    navdata.state = 2
    navdata.magX = 0
    navdata.magY = 0
    navdata.magZ = 0
    navdata.pressure = 0
    navdata.temp = 0
    navdata.wind_speed = 0.0
    navdata.wind_angle = 0.0
    navdata.wind_comp_angle = 0.0
    navdata.tags_count = 0

    navdata_pub.publish(navdata)
    rate.sleep()

Finally, we just create a loop in order to keep publishing this data constantly, updating the values with the current ones we read from the sensors (using the Subscribers you saw above). Inside the whole loop, we fill all the variables that the **Navdata** message contains. As you can see, there are many values that we are leaving as 0. You can try to fill these values if you want, but that is something I will leave to you.

So, all you have to do now is fill in the launch file to start this node. The launch file could be something like this:

**publish_navdata.launch**

In [ ]:
<launch>
    <node pkg="publish_navdata" type="publish_navdata_code.py" name="publish_navdata" output="screen">
        
    </node>
</launch>

Awesome!! So, you are now done! You can launch the code and check that everything works fine.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roslaunch publish_navdata publish_navdata.launch

Now, if you have a look at the topics in the simulation, you should see that a new topic has appeared, named **/hector_navdata**.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
rostopic list

You will see the new topic in the list.

<img src="img/hector_navdata.png" width="600" />

You can also have a look at what is being published into the topic.

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
rostopic echo /hector/navdata -n1

<img src="img/hector_navdata_echo.png" width="500" />

Awesome!! Now everything is ready and set up in order to use the **tum_ardrone** package with the hector_quadrotor drone. So, to conclude this chapter, just launch the tum_ardrone package and check that everything goes fine. You should be able to visualize  something like this in the Graphic Tools window.

<img src="img/PTAM_launched.png" width="800" />

<p style="background:#EE9023;color:white;">Exercise 3.1</p>
<br>
To successfully complete the chapter, set up the PTAM nodes as you learned to do in the previous chapter, and test that everything works fine.

<p style="background:#EE9023;color:white;">End of Exercise 2.5</p>